In [20]:
import pandas as pd
import requests
import os
import psycopg2
#import numpy as np

In [21]:
#Estadisticas de NBA
test_url = "https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=PerGame&Scope=S&Season=2022-23&SeasonType=Playoffs&StatCategory=PTS"
r =requests.get(url=test_url).json()


In [22]:
#Dar formatoa a la informacion obtenida
Columnas = r['resultSet']["headers"]
Tabla = pd.DataFrame(r['resultSet']["rowSet"],columns=Columnas)


In [23]:
#Hago chequeo de que no haya jugadores duplicados en la tabla
Tabla['PLAYER'].value_counts()

PLAYER
Devin Booker          1
Anthony Edwards       1
Davion Mitchell       1
Obi Toppin            1
Gary Payton II        1
                     ..
Bogdan Bogdanovic     1
Michael Porter Jr.    1
D'Angelo Russell      1
Aaron Gordon          1
Kessler Edwards       1
Name: count, Length: 144, dtype: int64

In [24]:
Tabla['PLAYER'].value_counts()

PLAYER
Devin Booker          1
Anthony Edwards       1
Davion Mitchell       1
Obi Toppin            1
Gary Payton II        1
                     ..
Bogdan Bogdanovic     1
Michael Porter Jr.    1
D'Angelo Russell      1
Aaron Gordon          1
Kessler Edwards       1
Name: count, Length: 144, dtype: int64

In [25]:
TablacheckC = Tabla
TablacheckC['PLAYER'].drop_duplicates(inplace=True)
TablacheckC['PLAYER'].value_counts()

PLAYER
Devin Booker          1
Anthony Edwards       1
Davion Mitchell       1
Obi Toppin            1
Gary Payton II        1
                     ..
Bogdan Bogdanovic     1
Michael Porter Jr.    1
D'Angelo Russell      1
Aaron Gordon          1
Kessler Edwards       1
Name: count, Length: 144, dtype: int64

In [26]:
#Genero 2 variables nuevas, saco apostrofes de los nombres para que no genere conflicto y selecciono los primeros 20 registros

Tabla['TEAM_PLAYERS'] = Tabla.groupby(['TEAM'])['TEAM'].transform('count')
Tabla['SUM_GP_TEAM'] = Tabla.groupby(['TEAM'])['GP'].transform('sum')
Tabla2 = Tabla.applymap(lambda x: x.replace("'", "") if (isinstance(x, str)) else x)
Tablaresumen = Tabla2[['PLAYER_ID', 'RANK','PLAYER', 'TEAM_ID', 'TEAM', 'GP', 'REB', 'AST', 'STL', 'BLK','TEAM_PLAYERS','SUM_GP_TEAM']]
Top20 = Tablaresumen[Tablaresumen["RANK"]<20]

In [27]:
Top20.head()

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,REB,AST,STL,BLK,TEAM_PLAYERS,SUM_GP_TEAM
0,1626164,1,Devin Booker,1610612756,PHX,11,4.8,7.2,1.7,0.8,8,79
1,1630162,2,Anthony Edwards,1610612750,MIN,5,5.0,5.2,1.8,2.0,9,43
2,203999,3,Nikola Jokic,1610612743,DEN,18,13.4,10.1,1.0,0.9,8,143
3,201939,4,Stephen Curry,1610612744,GSW,13,5.2,6.1,1.0,0.5,10,124
4,1629027,5,Trae Young,1610612737,ATL,6,3.7,10.2,1.7,0.7,9,53


In [41]:
from pyspark.sql import SparkSession
#from pyspark.sql.functions import when, lit, col

# Postgres and Redshift JDBCs
driver_path = "/home/coder/working_dir/driver_jdbc/postgresql-42.2.27.jre7.jar"

os.environ['PYSPARK_SUBMIT_ARGS'] = f'--driver-class-path {driver_path} --jars {driver_path} pyspark-shell'
os.environ['SPARK_CLASSPATH'] = driver_path

# Create SparkSession 
spark = SparkSession.builder \
        .master("local") \
        .appName("Conexion entre Pyspark y Redshift") \
        .config("spark.jars", driver_path) \
        .config("spark.executor.extraClassPath", driver_path) \
        .getOrCreate()

In [42]:
sparkDF=spark.createDataFrame(Top20) 
sparkDF.printSchema()
sparkDF.show()

root
 |-- PLAYER_ID: long (nullable = true)
 |-- RANK: long (nullable = true)
 |-- PLAYER: string (nullable = true)
 |-- TEAM_ID: long (nullable = true)
 |-- TEAM: string (nullable = true)
 |-- GP: long (nullable = true)
 |-- REB: double (nullable = true)
 |-- AST: double (nullable = true)
 |-- STL: double (nullable = true)
 |-- BLK: double (nullable = true)
 |-- TEAM_PLAYERS: long (nullable = true)
 |-- SUM_GP_TEAM: long (nullable = true)

+---------+----+-----------------+----------+----+---+----+----+---+---+------------+-----------+
|PLAYER_ID|RANK|           PLAYER|   TEAM_ID|TEAM| GP| REB| AST|STL|BLK|TEAM_PLAYERS|SUM_GP_TEAM|
+---------+----+-----------------+----------+----+---+----+----+---+---+------------+-----------+
|  1626164|   1|     Devin Booker|1610612756| PHX| 11| 4.8| 7.2|1.7|0.8|           8|         79|
|  1630162|   2|  Anthony Edwards|1610612750| MIN|  5| 5.0| 5.2|1.8|2.0|           9|         43|
|   203999|   3|     Nikola Jokic|1610612743| DEN| 18|13.4|10.1|1

In [43]:
env = os.environ

In [45]:
conn = psycopg2.connect(
    host=env['AWS_REDSHIFT_HOST'],
    port=env['AWS_REDSHIFT_PORT'],
    dbname=env['AWS_REDSHIFT_DBNAME'],
    user=env['AWS_REDSHIFT_USER'],
    password=env['AWS_REDSHIFT_PASSWORD']
)

In [46]:
cursor = conn.cursor()
cursor.execute(f"""
create table if not exists {env['AWS_REDSHIFT_SCHEMA']}.NBAstat2 (
    PLAYER_ID float distkey, 
    RANK float, PLAYER varchar,
    TEAM_ID float ,
    TEAM varchar,
    GP float,
    REB float,
    AST float,
    STL float,
    BLK float,
    TEAM_PLAYERS float,
     SUM_GP_TEAM float
    ) sortkey(TEAM);
""")
conn.commit()
cursor.close()
print("Table created!")

Table created!


In [47]:
sparkDF.write \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{env['AWS_REDSHIFT_HOST']}:{env['AWS_REDSHIFT_PORT']}/{env['AWS_REDSHIFT_DBNAME']}") \
    .option("dbtable", f"{env['AWS_REDSHIFT_SCHEMA']}.NBAstat2") \
    .option("user", env['AWS_REDSHIFT_USER']) \
    .option("password", env['AWS_REDSHIFT_PASSWORD']) \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()